In [ ]:
import numpy as np
import pandas as pd
from svmutil import *

In [ ]:
train = svm_read_problem("satimage.scale")
df = pd.DataFrame(train[1])
df = df.fillna(0)

In [37]:
trainX, trainY = df.to_numpy(), [1 if i == 3 else -1 for i in train[0]]

In [30]:
model = svm_train(trainY, trainX, "-s 0 -t 0 -c 10")

In [ ]:
index = [i-1 for i in model.get_sv_indices()]

In [ ]:
a = np.array(model.get_sv_coef())
x = df.loc[index].to_numpy()
y = np.array(trainY)[index].reshape(222, 1)

In [38]:
from sklearn.svm import SVC

model = SVC(C=10, kernel="linear", shrinking=False)
model = model.fit(trainX, trainY)

In [39]:
np.linalg.norm(model.coef_)
# model.n_support

9.869087082201911

In [ ]:
np.dot(x.T, (a * y))